In [1]:
import pandas as pd
import datetime as dt

In [2]:
gl = pd.read_parquet('../data/mine/gamelog_enhanced.parquet')

In [3]:
sps = gl[['game_id', 'visitor_starting_pitcher_id', 'home_starting_pitcher_id']]
sps

,game_id,visitor_starting_pitcher_id,home_starting_pitcher_id
73283,FW1187105040,prata101,mathb101
73284,WS3187105050,spala101,braia102
73285,RC1187105060,prata101,fishc102
73286,CH1187105080,prata101,zettg101
73287,TRO187105090,spala101,mcmuj101
...,...,...,...
20330,WAS201910250,greiz001,sanca004
20331,WAS201910260,urquj001,corbp001
20332,WAS201910270,coleg001,rossj002
20333,HOU201910290,stras001,verlj001


In [4]:
ppl = pd.read_parquet('../data/bd/people.parquet')[['retro_id', 'birth_year', 'birth_month', 'birth_day']].dropna()

In [5]:
def mk_birthdate(row):
    return dt.datetime(int(row['birth_year']), int(row['birth_month']), int(row['birth_day']))    

In [6]:
ppl['birth_date'] = ppl.apply(mk_birthdate, axis=1)
ppl

,retro_id,birth_year,birth_month,birth_day,birth_date
0,aardd001,1981.0,12.0,27.0,1981-12-27
1,aaroh101,1934.0,2.0,5.0,1934-02-05
2,aarot101,1939.0,8.0,5.0,1939-08-05
3,aased001,1954.0,9.0,8.0,1954-09-08
4,abada001,1972.0,8.0,25.0,1972-08-25
...,...,...,...,...,...
19873,zupof101,1939.0,8.0,29.0,1939-08-29
19874,zuvep001,1958.0,10.0,31.0,1958-10-31
19875,zuveg101,1924.0,8.0,20.0,1924-08-20
19876,zwild101,1888.0,11.0,2.0,1888-11-02


In [7]:
sps2 = pd.merge(left=sps,  right=ppl[['retro_id', 'birth_date']], left_on='visitor_starting_pitcher_id', right_on='retro_id').drop(columns='retro_id').rename(columns={'birth_date': 'visitor_starting_pitcher_birthdate'})
sps3 = pd.merge(left=sps2, right=ppl[['retro_id', 'birth_date']], left_on='home_starting_pitcher_id',    right_on='retro_id').drop(columns='retro_id').rename(columns={'birth_date':    'home_starting_pitcher_birthdate'})
sps_bd = sps3
sps_bd

,game_id,visitor_starting_pitcher_id,home_starting_pitcher_id,visitor_starting_pitcher_birthdate,home_starting_pitcher_birthdate
0,FW1187105040,prata101,mathb101,1847-11-19,1851-11-21
1,FW1187108110,prata101,mathb101,1847-11-19,1851-11-21
2,BL1187207090,prata101,mathb101,1847-11-19,1851-11-21
3,FW1187107120,spala101,mathb101,1850-09-02,1851-11-21
4,BL1187205020,spala101,mathb101,1850-09-02,1851-11-21
...,...,...,...,...,...
220208,SEA201906030,martc009,gearc001,1995-12-28,1986-04-14
220209,PHI201906230,yamaj001,deloe001,1996-05-11,1995-12-25
220210,LAN201909180,mckab001,sadlc002,1995-12-18,1990-07-13
220211,CHA201909260,civaa001,ruizj001,1995-06-12,1994-10-21


In [8]:
def sp_age_diff(row):
    return abs(row['visitor_starting_pitcher_birthdate'] - row['home_starting_pitcher_birthdate'])

In [9]:
sps_bd['age_diff'] = sps_bd.apply(sp_age_diff, axis=1)

In [10]:
sps_bd.sort_values(by='age_diff').tail(30)

,game_id,visitor_starting_pitcher_id,home_starting_pitcher_id,visitor_starting_pitcher_birthdate,home_starting_pitcher_birthdate,age_diff
189313,TOR201006270,moyej001,cecib001,1962-11-18,1986-07-02,8627 days
137803,NYA200709030,hernf002,clemr001,1986-04-08,1962-08-04,8648 days
145232,MIL198705220,niekp001,wegmb001,1939-04-01,1962-12-19,8663 days
145231,MIL198609050,niekp001,wegmb001,1939-04-01,1962-12-19,8663 days
188319,FLO200907160,moyej001,volsc001,1962-11-18,1986-09-23,8710 days
148616,PHI200905250,volsc001,moyej001,1986-09-23,1962-11-18,8710 days
188318,FLO200809210,moyej001,volsc001,1962-11-18,1986-09-23,8710 days
107325,NYA198504200,romaj001,niekp001,1963-05-21,1939-04-01,8816 days
143673,FLO199404160,torrs001,hougc001,1972-03-11,1948-01-05,8832 days
53063,DET192806130,quinj102,billj102,1883-07-01,1907-09-27,8853 days
